In [ ]:
import os
os.chdir(os.environ.get("GITHUB_WORKSPACE", os.getcwd()))

# ON GOOGLE COLAB, THIS SECTION MUST BE INITIALIZED. IT TAKES ABOUT 2MINUTES.
# AFTER RUNNING THIS SECTION, RUNTIME(SESSION) MUST BE RESTARTED, IN ORDER TO
# USE labwons/ PACKAGE. SESSION RESTART HOTKEY IS CTRL + M.
if any("COLAB" in key for key in os.environ):
    !git clone https://github.com/labwons/labwons.git
    %cd labwons
    !pip install -r requirements.txt -e .

In [6]:
ACTION = os.environ.get("GITHUB_EVENT_NAME", "LOCALHOST").upper()
HOSTID = os.environ.get("USERDOMAIN", "COLAB") if ACTION == "LOCALHOST" else "GITHUB"
JOBS   = [
    # "FETCH.DAILY-MARKET",
    # "FETCH.SECTOR-COMPOSITION",
    "FETCH.FINANCIAL-STATEMENT",
]

In [7]:
from labwons.util import DATETIME, DD

# GITHUB ACTION
if ACTION == "SCHEDULE":

    # SYSTEM EXITS WHEN THE LATEST TRADING DATE AND BUILD DATE IS DIFFERENT.
    # THIS CODE IS IMPLEMENTED IN ORDER TO AVOID RUNNING ON WEEKDAY OR HOLIDAY
    # OF THE MARKET.
    if DATETIME.TRADING != DATETIME.TODAY:
        raise SystemExit

    # IF THE TASK IS TRIGGERED BEFORE THE MARKET CLOSES, WHICH HARDLY HAPPENS,
    # SYSTEM PAUSES UNTIL THE MARKET CLOSES
    while (DATETIME.TIME().hour == 15) and (15 <= DATETIME.TIME().minute < 31):
        DATETIME.pause(30)

    if DATETIME.TIME().hour >= 20:
        JOBS = ["FETCH.ECOS", "FETCH.FRED", "FETCH.FINANCIAL-STATEMENT"]
    else:
        JOBS = ["FETCH.DAILY-MARKET", "FETCH.STOCKS"]
    JOBS += ["BUILD.MARKET-BASELINE", "BUILD.MACRO-BASELINE", "DEPLOY.ALL"]

# if ACTION == "LOCALHOST" and HOSTID == "HKEFICO":
#     JOBS = [job for job in JOBS if not job.startswith("FETCH")]

In [ ]:
from labwons.path import PATH
from labwons.fetch import (
    FETCH_OK, 
    DailyMarket, SectorComposition, FinancialStatement
)

archive = os.path.join(PATH.ARCHIVE, DATETIME.TODAY)
dataset = DD()
if "FETCH.DAILY-MARKET" in JOBS:
    dailyMarket = DailyMarket()
    if dailyMarket.status == FETCH_OK:
        dataset[os.path.join(archive, dailyMarket.fname)] = dailyMarket.data

if "FETCH.SECTOR-COMPOSITION" in JOBS:
    sectorComposition = SectorComposition()
    if sectorComposition.status == FETCH_OK:
        dataset[os.path.join(archive, sectorComposition.fname)] = sectorComposition.data

if "FETCH.FINANCIAL-STATEMENT" in JOBS:
    from pandas import read_parquet
    baseline = os.path.join(PATH.ARCHIVE, f"{max(int(date) for date in os.listdir(PATH.ARCHIVE))}/BASELINE.parquet")
    tickers = read_parquet(baseline, engine='pyarrow').index
    financialStatement = FinancialStatement(*tickers)
    if financialStatement.status == FETCH_OK:
        dataset[os.path.join(archive, 'OVERVIEW')] = financialStatement.overview
        dataset[os.path.join(archive, 'STATEMENT_A')] = financialStatement.annual
        dataset[os.path.join(archive, 'STATEMENT_Q')] = financialStatement.quarter

if dataset:
    os.makedirs(archive, exist_ok=True)
    for file, data in dataset.items():
        data.to_parquet(f"{file}.parquet", engine='pyarrow')


# TODO
# 일정 기간이 지난 날짜의 아카이브 폴더는 드라이브로 백업하는 코드 작성: Clean-UP

Index(['005930', '000660', '373220', '207940', '005380', '012450', '000270',
       '105560', '329180', '068270',
       ...
       '357230', '021820', '042520', '007980', '093920', '208370', '463480',
       '382150', '322180', '012860'],
      dtype='object', name='ticker', length=1361)


In [ ]:
from labwons.util import Mail
from labwons.logs import read_log

report = Mail()
report.content = content = read_log("fetch", "build")
report.subject = f'[{"FAILED" if "FAILED" in content else "SUCCESS"}] BUILD LABWONS : {DATETIME.CLOCK().strftime("%Y/%m/%d %H:%M")}'
report.send()

if HOSTID == "GITHUB":
    print(content)

'\n\n'